# E-Waste Management System using CNN
# This script generates synthetic e-waste data, trains a CNN to classify items based on images, and filters records for analysis.
# Tools: TensorFlow, Pandas, NumPy, Matplotlib, Keras

In [ ]:
# Importing libraries
import pandas as pd
import numpy as np
import random
import ast
import sys
import argparse
import gc
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping



# Step 1: Generate Mock Data
# Simulates e-waste items with image data and metadata.


In [ ]:

def generate_mock_data(sample_size=5000):
    try:
        categories = ['Computers', 'Mobiles', 'Batteries', 'Printers', 'Monitors',
                      'TVs', 'Cables', 'Scanners', 'Servers', 'Others']
        item_types = ['Laptop', 'Battery', 'Printer', 'Cable', 'Scanner', 'TV', 'Mobile']
        conditions = ['Working', 'Dead']
        sectors = ['IT', 'Telecom', 'Retail', 'Healthcare', 'Education', 'Manufacturing']

        data = []
        for _ in range(sample_size):
            data.append({
                "Item_Type": random.choice(item_types),
                "Weight(kg)": round(random.uniform(0.1, 20.0), 2),
                "Condition": random.choice(conditions),
                "Sector": random.choice(sectors),
                "Category": random.choice(categories),
                "Image_Data": np.random.rand(32, 32, 3).tolist()
            })

        df = pd.DataFrame(data)
        df.to_csv('e_waste_dataset.csv', index=False)
        print(f"Mock dataset created with {sample_size} records.")
        return df
    except Exception as e:
        print(f"Error during data generation: {e}")
        return None


# Step 2: Train CNN Model
# Prepares image data, builds a CNN, and evaluates performance.


In [ ]:

def train_cnn_model(df, epochs=10):
    try:
        print("Preparing image data for training...")
        X = []
        for idx, img in enumerate(df['Image_Data']):
            try:
                X.append(np.array(ast.literal_eval(str(img))))
            except Exception as e:
                print(f"Failed to parse image at index {idx}: {e}")
                return None, 0
        X = np.array(X)
        if X.size == 0:
            print("Image data could not be processed.")
            return None, 0
        le = LabelEncoder()
        y = le.fit_transform(df['Category'])
        y = to_categorical(y, num_classes=10)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        model = Sequential([
            Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
            MaxPooling2D((2, 2)),
            Conv2D(64, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Flatten(),
            Dense(128, activation='relu'),
            Dropout(0.5),
            Dense(10, activation='softmax')
        ])
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        early_stop = EarlyStopping(monitor='val_loss', patience=3)
        print("Training CNN model...")
        history = model.fit(
            X_train, y_train,
            epochs=epochs,
            batch_size=32,
            validation_split=0.2,
            callbacks=[early_stop],
            verbose=1
        )
        # Plot training progress
        plt.plot(history.history['accuracy'], label='Training Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title('Model Training Performance')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.grid(True)
        plt.savefig("training_accuracy.png")
        plt.close()
        print("Training accuracy graph saved as 'training_accuracy.png'.")

        loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
        print(f"Model training completed. Test Accuracy: {accuracy:.4f}")
        if accuracy >= 0.92:
            print("Target accuracy of 92% achieved.")
        else:
            print("Model did not reach target accuracy. Consider further tuning.")

        model.save('e_waste_cnn_classifier.h5')
        print("Model saved as 'e_waste_cnn_classifier.h5'.")

        del X_train, X_test, y_train, y_test
        gc.collect()

        return model, accuracy
    except Exception as e:
        print(f"Error during model training: {e}")
        return None, 0





#  Step 3: Filter and Export Data
# Filters non-working items above 1kg and exports them for visualization.


In [ ]:
def filter_data(df):
    try:
        filtered_df = df[(df['Condition'] == 'Dead') & (df['Weight(kg)'] > 1.0)]
        sector_counts = filtered_df.groupby('Sector').size().reset_index(name='Count')

        print("\nSector-wise breakdown of dead and heavy e-waste:")
        print(sector_counts)

        filtered_df.drop(columns=['Image_Data']).to_csv('filtered_data.csv', index=False)
        print("Filtered data saved to 'filtered_data.csv'.")
    except Exception as e:
        print(f"Error during data filtering: {e}")

#  Main Runner
# This block ties everything together using command-line arguments.


In [ ]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="E-Waste Management System")
    parser.add_argument('--samples', type=int, default=5000, help='Number of mock records to generate')
    parser.add_argument('--epochs', type=int, default=10, help='Epochs for training the CNN model')
    args = parser.parse_args()

    print("\nStarting E-Waste Management System...\n")

    if sys.version_info < (3, 7):
        print("This script requires Python 3.7 or higher.")
        sys.exit(1)

    df = generate_mock_data(sample_size=args.samples)
    if df is None:
        print("Data generation failed. Exiting.")
        sys.exit(1)

    model, accuracy = train_cnn_model(df, epochs=args.epochs)
    if model is None:
        print("Model training failed. Exiting.")
        sys.exit(1)

    filter_data(df)

    print("\nExecution complete.")
    print("Next steps:")
    print("- Open 'filtered_data.csv' in Tableau or Excel for analysis.")
    print("- Review 'training_accuracy.png' for model performance.")
    print("- Use 'e_waste_cnn_classifier.h5' to run predictions on new images.")